In [1]:
#不用print即可输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 导入库
**paddle.fluid**--->PaddlePaddle深度学习框架  
**numpy**---------->python基本库，用于科学计算  
**os**------------->python的模块，可使用该模块对操作系统进行操作  
**matplotlib**----->python绘图库，可方便绘制折线图、散点图等图形  

In [2]:
import paddle.fluid as fluid
import paddle
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

# 数据导入

In [16]:
task_path='data/enterprise_profit_forecast'
train=pd.read_csv('{}/output/train.csv'.format(task_path))
test=pd.read_csv('{}/output/test.csv'.format(task_path))

train_reader和test_reader

paddle.reader.shuffle()表示每次缓存BUF_SIZE个数据项，并进行打乱

paddle.batch()表示每BATCH_SIZE组成一个batch

In [39]:
for sam in train.values[:,1:]:
    sam
    break

array([ 8.000000e+00,  1.190000e+03,  3.000000e+00,  5.000000e+00,
        0.000000e+00,  1.000000e+00,  9.100000e-01,  1.000000e+00,
        0.000000e+00,  0.000000e+00,  3.332000e+03,  2.665600e+02,
        0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
        0.000000e+00,  0.000000e+00,  4.760000e+03,  3.808000e+02,
        0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
        0.000000e+00,  0.000000e+00,  7.400000e+01,  3.332000e+04,
        6.426000e+04,  1.466080e+05,  1.026256e+05,  7.330400e+04,
       -1.466080e+04,  0.000000e+00, -3.094000e+04,  6.310000e+02,
        4.760000e+04,  6.961500e+04,  5.236000e+04,  2.094400e+04,
        1.570800e+04, -5.236000e+03,  0.000000e+00, -2.201500e+04,
        8.353800e+03])

In [36]:
def reader(df):
    data=df.values[:,1:]
    for sample in data:
        yield sample

In [45]:
def splitData(df):
    train=df.sample(frac=0.8)
    test=df[~df.index.isin(train.index)]
    return train.values[:,1:],test.values[:,1:]

In [46]:
train_data,test_data=splitData(train)

In [52]:
train_data=train_data.reshape(-1,1)
test_data=test_data.reshape(-1,1)
train_data.shape

(623475, 1)

In [55]:
def reader(data):
    for sample in data:
        yield sample

In [58]:
BUF_SIZE=500
BATCH_SIZE=20

#用于训练的数据提供器，每次从缓存中随机读取批次大小的数据
train_reader=paddle.batch(fluid.io.shuffle(reader(train_data),buf_size=BUF_SIZE),batch_size=BATCH_SIZE)
test_reader=paddle.batch(fluid.io.shuffle(reader(test_data),buf_size=BUF_SIZE),batch_size=BATCH_SIZE)

In [18]:
train.shape
test.shape

(17319, 46)

(4068, 45)

# 网络配置

#### 网络搭建
对于线性回归来讲，它就是一个从输入到输出的简单的全连接层

![https://aistudio.baidu.com/aistudio/projectdetail/78917](https://ai-studio-static-online.cdn.bcebos.com/f3f567e7aa7a499fb3abb767aaaa1b943eed373cf4694d87beb382e161ea8edc)

![https://aistudio.baidu.com/aistudio/projectdetail/78917](https://ai-studio-static-online.cdn.bcebos.com/f6c44a96e1624828829bbd438c29c17ead9ecc45c68b4310bfbbb0a4dc96c3fe)

In [19]:
#定义张量变量x，表示44维的特征值
x = fluid.layers.data(name='x', shape=[44], dtype='float32')
#定义张量y，表示目标值
y = fluid.layers.data(name='y', shape=[1], dtype='float32')
#定义一个简单的线性网络,连接输入和输出的全连接层
#input:输入tensor;
#size:该层输出单元的数目
#act:激活函数
y_predict=fluid.layers.fc(input=x,size=1,act=None)

#### 定义损失函数
此处使用均方差损失函数。

square_error_cost(input,lable):接受输入预测值和目标值，并返回方差估计,即为（y-y_predict）的平方

In [20]:
cost = fluid.layers.square_error_cost(input=y_predict, label=y) #求一个batch的损失值
avg_cost = fluid.layers.mean(cost)                              #对损失值求平均值

#### 定义优化函数

In [21]:
optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.001)
opts = optimizer.minimize(avg_cost)

In [22]:
test_program = fluid.default_main_program().clone(for_test=True)

在上述模型配置完毕后，得到两个fluid.Program：fluid.default_startup_program() 与fluid.default_main_program() 配置完毕了。

参数初始化操作会被写入**fluid.default_startup_program()**

**fluid.default_main_program()**用于获取默认或全局main program(主程序)。该主程序用于训练和测试模型。fluid.layers 中的所有layer函数可以向 default_main_program 中添加算子和变量。default_main_program 是fluid的许多编程接口（API）的Program参数的缺省值。例如,当用户program没有传入的时候， Executor.run() 会默认执行 default_main_program 。

# 模型训练与评估

#### 创建Executor
首先定义运算场所 fluid.CPUPlace()和 fluid.CUDAPlace(0)分别表示运算场所为CPU和GPU

Executor:接收传入的program，通过run()方法运行program。

In [24]:
use_cuda = True                         #use_cuda为False,表示运算场所为CPU;use_cuda为True,表示运算场所为GPU           
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(place)              #创建一个Executor实例exe
exe.run(fluid.default_startup_program()) #Executor的run()方法执行startup_program(),进行参数初始化

[]

#### 定义输入数据维度
DataFeeder负责将数据提供器（train_reader,test_reader）返回的数据转成一种特殊的数据结构，使其可以输入到Executor中。

feed_list设置向模型输入的向变量表或者变量表名

In [25]:
# 定义输入数据维度
feeder = fluid.DataFeeder(place=place, feed_list=[x, y])#feed_list:向模型输入的变量表或变量表名

#### 定义绘制训练过程的损失值变化趋势的方法draw_train_process

In [26]:
iter=0;
iters=[]
train_costs=[]

def draw_train_process(iters,train_costs):
    title="training cost"
    plt.title(title, fontsize=24)
    plt.xlabel("iter", fontsize=14)
    plt.ylabel("cost", fontsize=14)
    plt.plot(iters, train_costs,color='red',label='training cost') 
    plt.grid()
    plt.show()

#### 训练并保存模型
Executor接收传入的program,并根据feed map(输入映射表)和fetch_list(结果获取表) 向program中添加feed operators(数据输入算子)和fetch operators（结果获取算子)。 feed map为该program提供输入数据。fetch_list提供program训练结束后用户预期的变量。

注：enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标

In [59]:
EPOCH_NUM=50
model_save_dir = "data/enterprise_profit_forecast/model/fit_a_line.inference.model"

for pass_id in range(EPOCH_NUM):                                  #训练EPOCH_NUM轮
    # 开始训练并输出最后一个batch的损失值
    train_cost = 0
    for batch_id, data in enumerate(train_reader()):              #遍历train_reader迭代器
        train_cost = exe.run(program=fluid.default_main_program(),#运行主程序
                             feed=feeder.feed(data),              #喂入一个batch的训练数据，根据feed_list和data提供的信息，将输入数据转成一种特殊的数据结构
                             fetch_list=[avg_cost])    
        if batch_id % 40 == 0:
            print("Pass:%d, Cost:%0.5f" % (pass_id, train_cost[0][0]))    #打印最后一个batch的损失值
        iter=iter+BATCH_SIZE
        iters.append(iter)
        train_costs.append(train_cost[0][0])
       
   
    # 开始测试并输出最后一个batch的损失值
    test_cost = 0
    for batch_id, data in enumerate(test_reader()):               #遍历test_reader迭代器
        test_cost= exe.run(program=test_program, #运行测试cheng
                            feed=feeder.feed(data),               #喂入一个batch的测试数据
                            fetch_list=[avg_cost])                #fetch均方误差
    print('Test:%d, Cost:%0.5f' % (pass_id, test_cost[0][0]))     #打印最后一个batch的损失值
    
    #保存模型
    # 如果保存路径不存在就创建
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)
print ('save models to %s' % (model_save_dir))
#保存训练参数到指定路径中，构建一个专门用预测的program
fluid.io.save_inference_model(model_save_dir,   #保存推理model的路径
                                  ['x'],            #推理（inference）需要 feed 的数据
                                  [y_predict],      #保存推理（inference）结果的 Variables
                                  exe)              #exe 保存 inference model
draw_train_process(iters,train_costs)

TypeError: 'generator' object is not callable